## Predicting Survival on the Titanic

### History
Perhaps one of the most infamous shipwrecks in history, the Titanic sank after colliding with an iceberg, killing 1502 out of 2224 people on board. Interestingly, by analysing the probability of survival based on few attributes like gender, age, and social status, we can make very accurate predictions on which passengers would survive. Some groups of people were more likely to survive than others, such as women, children, and the upper-class. Therefore, we can learn about the society priorities and privileges at the time.

### Assignment:

Build a Machine Learning Pipeline, to engineer the features in the data set and predict who is more likely to Survive the catastrophe.

Follow the Jupyter notebook below, and complete the missing bits of code, to achieve each one of the pipeline steps.

In [351]:
import re

# to handle datasets
import pandas as pd
import numpy as np

# for visualization
import matplotlib.pyplot as plt

# to divide train and test set
from sklearn.model_selection import train_test_split

# feature scaling
from sklearn.preprocessing import StandardScaler

# to build the models
from sklearn.linear_model import LogisticRegression

# to evaluate the models
from sklearn.metrics import accuracy_score, roc_auc_score

# to persist the model and the scaler
import joblib

# to visualise al the columns in the dataframe
pd.pandas.set_option('display.max_columns', None)

## Prepare the data set

In [352]:
# load the data - it is available open source and online

data = pd.read_csv('https://www.openml.org/data/get_csv/16826755/phpMYEkMl')

# display data
data.head()
# data.shape

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29,0,0,24160,211.3375,B5,S,2,?,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.55,C22 C26,S,11,?,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2,1,2,113781,151.55,C22 C26,S,?,?,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30,1,2,113781,151.55,C22 C26,S,?,135,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25,1,2,113781,151.55,C22 C26,S,?,?,"Montreal, PQ / Chesterville, ON"


In [353]:
# replace interrogation marks by NaN values

data = data.replace('?', np.nan)

In [354]:
# retain only the first cabin if more than
# 1 are available per passenger

def get_first_cabin(row):
    try:
        return row.split()[0]
    except:
        return np.nan
    
data['cabin'] = data['cabin'].apply(get_first_cabin)

In [355]:
# extracts the title (Mr, Ms, etc) from the name variable

def get_title(passenger):
    line = passenger
    if re.search('Mrs', line):
        return 'Mrs'
    elif re.search('Mr', line):
        return 'Mr'
    elif re.search('Miss', line):
        return 'Miss'
    elif re.search('Master', line):
        return 'Master'
    else:
        return 'Other'
    
data['title'] = data['name'].apply(get_title)

In [356]:
# cast numerical variables as floats

data['fare'] = data['fare'].astype('float')
data['age'] = data['age'].astype('float')

In [357]:
# drop unnecessary variables

data.drop(labels=['name','ticket', 'boat', 'body','home.dest'], axis=1, inplace=True)

# display data
data.head()

,pclass,survived,sex,age,sibsp,parch,fare,cabin,embarked,title
0,1,1,female,29.0000,0,0,211.3375,B5,S,Miss
1,1,1,male,0.9167,1,2,151.5500,C22,S,Master
2,1,0,female,2.0000,1,2,151.5500,C22,S,Miss
3,1,0,male,30.0000,1,2,151.5500,C22,S,Mr
4,1,0,female,25.0000,1,2,151.5500,C22,S,Mrs


In [358]:
# save the data set

data.to_csv('titanic.csv', index=False)

## Data Exploration

### Find numerical and categorical variables

In [359]:
target = 'survived'

In [360]:
# vars_num = data.select_dtypes(include=np.number).columns.tolist()# fill your code here
vars_num = [var for var in data.select_dtypes(include=np.number) if var != target]

vars_cat = [var for var in data.columns if var not in vars_num and var != target]# fill your code here

print('Number of numerical variables: {}'.format(len(vars_num)))
print('Number of categorical variables: {}'.format(len(vars_cat)))

Number of numerical variables: 5
Number of categorical variables: 4


### Find missing values in variables

In [361]:
# first in numerical variables
data[vars_num].isnull().mean().sort_values(ascending=False)


age       0.200917
fare      0.000764
pclass    0.000000
sibsp     0.000000
parch     0.000000
dtype: float64

In [362]:
# now in categorical variables

data[vars_cat].isnull().mean().sort_values(ascending=False)

cabin       0.774637
embarked    0.001528
sex         0.000000
title       0.000000
dtype: float64

### Determine cardinality of categorical variables

In [363]:
data[vars_cat].nunique()

sex           2
cabin       181
embarked      3
title         5
dtype: int64

### Determine the distribution of numerical variables

In [364]:
data[vars_num].describe()#[data[var].value_counts() for var in vars_num]

,pclass,age,sibsp,parch,fare
count,1309.000000,1046.000000,1309.000000,1309.000000,1308.000000
mean,2.294882,29.881135,0.498854,0.385027,33.295479
std,0.837836,14.413500,1.041658,0.865560,51.758668
min,1.000000,0.166700,0.000000,0.000000,0.000000
25%,2.000000,21.000000,0.000000,0.000000,7.895800
50%,3.000000,28.000000,0.000000,0.000000,14.454200
75%,3.000000,39.000000,1.000000,0.000000,31.275000
max,3.000000,80.000000,8.000000,9.000000,512.329200


## Separate data into train and test

Use the code below for reproducibility. Don't change it.

In [365]:
X_train, X_test, y_train, y_test = train_test_split(
    data.drop('survived', axis=1),  # predictors
    data['survived'],  # target
    test_size=0.2,  # percentage of obs in test set
    random_state=0)  # seed to ensure reproducibility

X_train.shape, X_test.shape

((1047, 9), (262, 9))

## Feature Engineering

### Extract only the letter (and drop the number) from the variable Cabin

In [366]:
X_train['cabin'] = X_train['cabin'].str[0]
X_test['cabin'] = X_test['cabin'].str[0]

### Fill in Missing data in numerical variables:

- Add a binary missing indicator
- Fill NA in original variable with the median

In [367]:
for var in vars_num:
    X_train[var + '_na'] = np.where(X_train[var].isnull(), 1, 0)
    X_test[var + '_na'] = np.where(X_test[var].isnull(), 1, 0)
    X_train[var].fillna(X_train[var].mean(), inplace=True)
    X_test[var].fillna(X_train[var].mean(), inplace=True)
X_train[vars_num].isnull().mean().sort_values(ascending=False)
X_test[vars_num].isnull().mean().sort_values(ascending=False)

pclass    0.0
age       0.0
sibsp     0.0
parch     0.0
fare      0.0
dtype: float64

### Replace Missing data in categorical variables with the string **Missing**

In [368]:
for var in vars_cat:
    X_train[var].fillna("Missing", inplace=True)
    X_test[var].fillna("Missing", inplace=True)

In [369]:
X_train[vars_cat].isnull().mean().sort_values(ascending=False)

sex         0.0
cabin       0.0
embarked    0.0
title       0.0
dtype: float64

In [370]:
X_test[vars_cat].isnull().mean().sort_values(ascending=False)

sex         0.0
cabin       0.0
embarked    0.0
title       0.0
dtype: float64

### Remove rare labels in categorical variables

- remove labels present in less than 5 % of the passengers

In [371]:
rare_perc = 0.05

In [372]:
def find_frequent_labels(data, var, rare_perc):
    data = data.copy()
    tmp = data.groupby(var)[var].count()/len(data)
    return tmp[tmp >= rare_perc].index

In [373]:
for var in vars_cat:
    freq_labels = find_frequent_labels(data, var, rare_perc)
    print(var, freq_labels)
    X_train[var] = np.where(X_train[var].isin(freq_labels), X_train[var], 'Rare')
    X_test[var] = np.where(X_test[var].isin(freq_labels), X_test[var], 'Rare')

sex Index(['female', 'male'], dtype='object', name='sex')
cabin Index([], dtype='object', name='cabin')
embarked Index(['C', 'Q', 'S'], dtype='object', name='embarked')
title Index(['Miss', 'Mr', 'Mrs'], dtype='object', name='title')


### Perform one hot encoding of categorical variables into k-1 binary variables

- k-1, means that if the variable contains 9 different categories, we create 8 different binary variables
- Remember to drop the original categorical variable (the one with the strings) after the encoding

In [374]:
from feature_engine.encoding import OneHotEncoder
encoding = OneHotEncoder(drop_last=True)
X_train = encoding.fit_transform(X_train)
X_test = encoding.transform(X_test)
X_train, X_test

(      pclass        age  sibsp  parch      fare  pclass_na  age_na  sibsp_na  \
 1118       3  25.000000      0      0    7.9250          0       0         0   
 44         1  41.000000      0      0  134.5000          0       0         0   
 1072       3  30.051823      0      0    7.7333          0       1         0   
 1130       3  18.000000      0      0    7.7750          0       0         0   
 574        2  29.000000      1      0   21.0000          0       0         0   
 ...      ...        ...    ...    ...       ...        ...     ...       ...   
 763        3   0.166700      1      2   20.5750          0       0         0   
 835        3  30.051823      0      0    8.0500          0       1         0   
 1216       3  30.051823      0      0    7.7333          0       1         0   
 559        2  20.000000      0      0   36.7500          0       0         0   
 684        3  32.000000      1      1   15.5000          0       0         0   
 
       parch_na  fare_na  

In [375]:
X_train.columns, X_test.columns

(Index(['pclass', 'age', 'sibsp', 'parch', 'fare', 'pclass_na', 'age_na',
        'sibsp_na', 'parch_na', 'fare_na', 'sex_male', 'embarked_S',
        'embarked_C', 'embarked_Q', 'title_Mr', 'title_Miss', 'title_Mrs'],
       dtype='object'),
 Index(['pclass', 'age', 'sibsp', 'parch', 'fare', 'pclass_na', 'age_na',
        'sibsp_na', 'parch_na', 'fare_na', 'sex_male', 'embarked_S',
        'embarked_C', 'embarked_Q', 'title_Mr', 'title_Miss', 'title_Mrs'],
       dtype='object'))

In [376]:
X_train.head()

,pclass,age,sibsp,parch,fare,pclass_na,age_na,sibsp_na,parch_na,fare_na,sex_male,embarked_S,embarked_C,embarked_Q,title_Mr,title_Miss,title_Mrs
1118,3,25.000000,0,0,7.9250,0,0,0,0,0,1,1,0,0,1,0,0
44,1,41.000000,0,0,134.5000,0,0,0,0,0,0,0,1,0,0,1,0
1072,3,30.051823,0,0,7.7333,0,1,0,0,0,1,0,0,1,1,0,0
1130,3,18.000000,0,0,7.7750,0,0,0,0,0,0,1,0,0,0,1,0
574,2,29.000000,1,0,21.0000,0,0,0,0,0,1,1,0,0,1,0,0


In [377]:
assert list(X_train.columns)==list(X_test.columns)

### Scale the variables

- Use the standard scaler from Scikit-learn

In [378]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = pd.DataFrame(
    scaler.transform(X_train),
    columns=X_train.columns
)
X_test = pd.DataFrame(
    scaler.transform(X_test),
    columns=X_train.columns
)


## Train the Logistic Regression model

- Set the regularization parameter to 0.0005
- Set the seed to 0

In [379]:
lg_model = LogisticRegression(C=0.0005, random_state=0)
lg_model.fit(X_train, y_train)

LogisticRegression(C=0.0005, random_state=0)

## Make predictions and evaluate model performance

Determine:
- roc-auc
- accuracy

**Important, remember that to determine the accuracy, you need the outcome 0, 1, referring to survived or not. But to determine the roc-auc you need the probability of survival.**

In [380]:
y_pred = lg_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"{accuracy=}")
y_prob = lg_model.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_prob)
print(f"{roc_auc=}")

accuracy=0.6870229007633588
roc_auc=0.8319444444444445


That's it! Well done

**Keep this code safe, as we will use this notebook later on, to build production code, in our next assignement!!**

In [381]:
joblib.dump(lg_model, 'logistic_regression.joblib') 

['logistic_regression.joblib']